In [5]:
import pandas as pd
import librosa
import numpy as np
from concurrent.futures import ProcessPoolExecutor

In [2]:
# Load metadata
metadata = pd.read_csv('/home/saashiv/Code/EMR/data/audio_data/UrbanSound8K/metadata/UrbanSound8K.csv')
emergency_classes = ['dog_bark', 'car_horn', 'gun_shot','siren','jackhammer','drilling']  # Add more classes as needed
filtered_metadata = metadata[metadata['class'].isin(emergency_classes)]

In [9]:
def feature_extraction(index_row):
    index, row = index_row
    if index%100==0:
        print(f"Processing index: {index}, class: {row['class']}")
    file_path = f'/home/saashiv/Code/EMR/data/audio_data/UrbanSound8K/audio/fold{row["fold"]}/{row["slice_file_name"]}'
    class_label = row['classID']  # or row['class'] if you want the string label
    
    # Load audio file
    audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    
    # Extract MFCC features
    mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)
    return mfccs,class_label

In [10]:
# Load metadata
metadata = None
filtered_metadata = None
metadata = pd.read_csv('/home/saashiv/Code/EMR/data/audio_data/UrbanSound8K/metadata/UrbanSound8K.csv')
emergency_classes = ['dog_bark', 'car_horn', 'gun_shot','siren','jackhammer','drilling']  # Add more classes as needed
filtered_metadata = metadata[metadata['class'].isin(emergency_classes)]
filtered_metadata.reset_index(drop=True, inplace=True)
print(len(filtered_metadata))

X = []
y = []

with ProcessPoolExecutor() as executor:
    results = list(executor.map(feature_extraction, filtered_metadata.iterrows()))
    
for mfccs, class_label in results:
    X.append(mfccs)
    y.append(class_label)

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

4732
Processing index: 0, class: dog_bark
Processing index: 100, class: drilling
Processing index: 200, class: jackhammer
Processing index: 300, class: siren
Processing index: 400, class: dog_bark
Processing index: 500, class: siren
Processing index: 600, class: jackhammer
Processing index: 700, class: jackhammer
Processing index: 800, class: gun_shot
Processing index: 900, class: drilling
Processing index: 1000, class: siren
Processing index: 1100, class: drilling
Processing index: 1200, class: siren
Processing index: 1300, class: gun_shot
Processing index: 1400, class: siren
Processing index: 1500, class: drilling
Processing index: 1600, class: gun_shot
Processing index: 1700, class: drilling
Processing index: 1800, class: jackhammer


/home/saashiv/miniconda3/envs/ERP/lib/python3.8/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(


Processing index: 1900, class: dog_bark
Processing index: 2000, class: jackhammer
Processing index: 2100, class: dog_bark
Processing index: 2200, class: drilling
Processing index: 2300, class: jackhammer
Processing index: 2400, class: car_horn
Processing index: 2500, class: jackhammer
Processing index: 2600, class: dog_bark
Processing index: 2700, class: jackhammer
Processing index: 2800, class: dog_bark
Processing index: 2900, class: jackhammer
Processing index: 3000, class: siren
Processing index: 3100, class: dog_bark
Processing index: 3200, class: jackhammer
Processing index: 3300, class: drilling
Processing index: 3400, class: siren
Processing index: 3500, class: drilling
Processing index: 3600, class: dog_bark
Processing index: 3700, class: drilling
Processing index: 3800, class: jackhammer
Processing index: 3900, class: siren
Processing index: 4000, class: drilling
Processing index: 4100, class: car_horn
Processing index: 4200, class: jackhammer
Processing index: 4300, class: si

/home/saashiv/miniconda3/envs/ERP/lib/python3.8/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
/home/saashiv/miniconda3/envs/ERP/lib/python3.8/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
/home/saashiv/miniconda3/envs/ERP/lib/python3.8/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(
/home/saashiv/miniconda3/envs/ERP/lib/python3.8/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(


Processing index: 4600, class: siren
Processing index: 4700, class: drilling


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assume X is your feature matrix and y is your label vector
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9165786694825766


In [20]:
random_row = filtered_metadata.sample(1).iloc[0]
print(random_row)

slice_file_name    137815-4-0-7.wav
fsID                         137815
start                      3.706472
end                        7.706472
salience                          1
fold                              9
classID                           4
class                      drilling
Name: 851, dtype: object


In [21]:
file_path = f'/home/saashiv/Code/EMR/data/audio_data/UrbanSound8K/audio/fold{random_row["fold"]}/{random_row["slice_file_name"]}'
audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)
mfccs = mfccs.reshape(1, -1)  # Reshape to match the expected input shape for the classifier
predicted_label = clf.predict(mfccs)
print("Predicted Label:", predicted_label, 'actual label:',random_row['class'])

Predicted Label: [4] actual label: drilling


In [23]:
import joblib

# Save the model
joblib.dump(clf, '/home/saashiv/Code/EMR/ML models/audio_model/ML_audio_model.joblib')

['/home/saashiv/Code/EMR/ML models/audio_model/ML_audio_model.joblib']

In [ ]:
loaded_model = joblib.load('random_forest_audio_model.joblib')

# To use the loaded model to make predictions
# predictions = loaded_model.predict(X_test)
